In [1]:
# Import packages

import pandas as pd 
import numpy as np
import re

import nltk
from nltk import word_tokenize as w_tokenize
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
from nltk.corpus import stopwords
import string
stop_words = set(stopwords.words('english') + ["..."])

from sklearn.feature_extraction.text import TfidfVectorizer as tfv
from sklearn.decomposition import NMF as nmf

import pdfminer
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.high_level import extract_text
import PyPDF2
# !pip install PyPDF2

In [2]:
# extract text from list of pdf titles and create dataframe of text

path = './Scripts/'
end = '.pdf'
titles = ['1917', '20th_Century_Women', 'BridgeofSpies', 'ExMachina', 'First_Reformed',
         'Get_Out', 'GreenBook', 'Hell_or_HighWater', 'InsideOut', 'KnivesOut', 'LadyBird',
         'LaLaLand', 'Manchester_By_TheSea', 'MarriageStory', 'Parasite', 'Roma', 
          'ShapeofWater', 'Spotlight', 'StraightOuttaCompton', 'TheBigSick', 
         'TheFavourite', 'TheLobster', 'ThreeBillboards', 'Vice']

movie_scripts = []
for title in titles:
    script = extract_text(path + title + end)
    movie_scripts.append(script)
#     print (title)

# create dataframe of scripts/titles from lists    
movie_df = pd.DataFrame(list(zip(titles, movie_scripts)), columns = ['Title', 'Script'])

In [3]:
# add label for original screenplay win, best picture win, and year
orig_screen_win = [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
best_pic_win = ['Nom', None, 'Nom', None, None, 'Nom', 'Win', 'Nom', None, None, 'Nom', 'Nom', 'Nom', 'Nom', 'Win', 'Nom', 'Win', 'Win', None, None, 'Nom', None, 'Nom', 'Nom']
year = [2019, 2016, 2015, 2015, 2018, 2017, 2018, 2016, 2015, 2019, 2017, 2016, 2016, 2019, 2019, 2018, 2017, 2015, 2015, 2017, 2018, 2016, 2017, 2018]
reviewer = ["Shafer", "Shafer", "Shafer", "Shafer", "Ragan", "Shafer", "Ragan", "Ragan", "Shafer", "Shafer", "Ragan", "Ragan", "Shafer", "Ragan", "Shafer", "Ragan", "Ragan", "Shafer", "Shafer", "Ragan", "Ragan", "Shafer", "Ragan", "Ragan"]

# ADD GENRE

# sanity check -- should return True
len(orig_screen_win) == len(year) == len(best_pic_win) ==  len(list(movie_df['Title'])) == len(reviewer)

movie_df['year'] = year
movie_df['orig_screen_win'] = orig_screen_win
movie_df['best_pic_win'] = best_pic_win
movie_df['reviewer'] = reviewer

In [4]:
# movie_df

# column of dialogue
# column of visual cues
# column of setting
# raw script column
# everything column


# corpus of just dialogue
# corpus of everything (dialogue, settings, visual cues, etc.) cleaned up
# corpus of visual cues

# so 3 x per model
    # sentiment analysis on dialogue (CV + TFIDF)
    # sentiment analysis on everything (CV + TFIDF)
    # sentimental analysis on visual cues
    # and so on forth

# POS tagging and keep only proper names and put rest into settings (when dialogue vs setting can both be caps) 

In [32]:
ar = movie_df[movie_df['reviewer'] == "Ragan"].copy()
ar['CleanedScript'] = ar['Script']

raw_scripts = list(ar['Script'])

##### semi-manually remove extraneous beginning junk

varies by script, easier to do it manually than write a function with 10k if statements that may still not capture everything

In [34]:
def drop_start(idx, todrop):
    # return script without extraneous beginning stuff
    return raw_scripts[idx].replace(todrop, "")

extraneous = ["Scripts.com\n\nFirst Reformed\n\nBy Paul Schrader\n\nPage 1/57\n\n\x0c", 
              "GREEN BOOK \n\nWritten by \n\nNick Vallelonga & Brian Currie & Peter Farrelly \n\n\x0c", 
              "Scripts.com\n\nHell or High Water\n\nBy Taylor Sheridan\n\nPage 1/42\n\n\x0c", 
              " \n \n\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n\n \n \n \n \n \n \n \n \n \n \n \n\nLADY BIRD \n\nwritten by \n\nGreta Gerwig \n\n\x0c \n \n\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n\nii. \n\n", 
              "LA LA LAND\n\nby\n\nDamien Chazelle\n\n\x0c", 
              "MARRIAGE STORY\n\nWritten and Directed by Noah Baumbach\n\n\x0cBlack.\n\n", 
              "IN ENGLISH\n\n\x0cROMA\n\nWritten and Directed by\n\nAlfonso Cuarón\n\nDates in RED are meant only as a tool for the different departments for \nthe specific historical accuracy of the scenes and are not intended to \nappear on screen. \n\n\x0cThursday, September 3rd, 1970\n\n", 
              "F O R   Y O U R   C O N S I D E R A T I O N\n\nB E S T   O R I G I N A L   S C R E E N P L A Y\n\nG U I L L E R M O   D E L   T O R O\n&\nV A N E S S A   T A Y L O R\n\n\x0c1                             \n\n1\n\n", 
              "THE BIG SICK\n\nby\nEmily V. Gordon & Kumail Nanjiani\n\n\x0c1\n\n", 
              "F O R   Y O U R   C O N S I D E R A T I O N\n\nW R I T T E N   B Y\n\nD E B O R A H   D A V I S   A N D   T O N Y   M C N A M A R A\n\n\x0cW R I T T E N   B Y\n\nD E B O R A H   D A V I S   A N D   T O N Y   M C N A M A R A\n\n\x0cTHE FAVOURITE\n\nWritten by\n\nDeborah Davis and Tony McNamara\n\nFINAL SHOOTING SCRIPT - 23rd MARCH 2017\n\nElement Pictures\n21 Mespil Road \nDublin 4\nIreland\n\nScarlet Films\n3 Oakley Studios\nUpper Cheyne Row\nLondon SW3 5JW, UK\n\n \n\n\x0c1\n\n2\n\n3\n\n4\n\n5\n\n6\n\n", 
              "F O R   Y O U R   C O N S I D E R A T I O N\n\nB E S T   O R I G I N A L   S C R E E N P L A Y\n\nM A R T I N   M c D O N A G H\n\n\x0cTHREE BILLBOARDS OUTSIDE EBBING, MISSOURI\n\nYou Red Welby?\n\nby\nMartin McDonagh\n\n1\n\n", 
              "Written and Directed by\n\nVICE\n\nAdam McKay\n\n\x0c"]

for r in range(len(raw_scripts)):
    raw_scripts[r] = drop_start(r, extraneous[r])

# expand contractions
def expand_contractions(text):
    text = re.sub("\'s", " is", text) # it's --> it is
    text = re.sub("n\'t", " not", text) # don't --> do not
    text = re.sub("\'re", " are", text) # you're --> you are
    text = re.sub("\'d", " would", text) # she'd --> she would
    text = re.sub("\'ll", " will", text) # he'll --> he will
    text = re.sub("\'ve", " have", text) # we've --> we have
    text = re.sub("\'m", " am", text) # I'm --> I am
    return text

for i in range(len(raw_scripts)):
    raw_scripts[i] = expand_contractions(raw_scripts[i])

# instantiate empty lists

master_visual_cues = []

master_dialogue = []

master_setting = []

master_cleaned_everything = []

##### process First Reformed

Speaking: all caps NAME \n text	
Setting: none, script is just dialogue and a few visuals	
Character: bold all caps:	
Visual: (ALL CAPS)
Other Notes: some lines are randomly preceded by dashes and includes page numbers

In [35]:
# drop page numbers
raw_scripts[0] = re.sub("\\nPage \d+/57", " ", raw_scripts[0])

# extract visual cues from First Reformed
# visual cues are in ()
# tokenize
toked_fr = w_tokenize(raw_scripts[0])
orig_toked_fr = toked_fr.copy()
# find indexes with (
open_pars = [i for i, e in enumerate(toked_fr) if e == "("]
# find indexes with )
end_pars = [i for i, e in enumerate(toked_fr) if e == ")"]
# return words between ( and ) to extract visual cues
visual_cues_fr = [" ".join(toked_fr[open_pars[o]+1:end_pars[o]]) for o in range(len(open_pars))]

# drop visual cues from list
for o in range(len(open_pars)):
    del toked_fr[open_pars[o]:end_pars[0]+1]

# re-join : with character names and drop extra :
# find indices with :
char_idx = [i for i, item in enumerate(toked_fr) if item.endswith(':')]
# for every index with :, go to the index before and add : to re-join CHARACTER with :
for b in char_idx:
    toked_fr[b-1] = toked_fr[b-1] + toked_fr[b]
# drop extraneous colons
toked_fr.remove(":")

# append non-character names to dialogue list
dialogue_fr = [toked_fr[t] for t in range(len(toked_fr)) if toked_fr[t].endswith(":") == False]

def clean_lists(lis):
    # strip punctuation
    lis = [t.lower() for t in lis if t not in set(string.punctuation)]
    # lemmatize + lowercase
    lis = [lemma.lemmatize(l) for l in lis]
    # drop stop words
    lis = [t for t in lis if t not in set(stop_words)]
    return lis

# clean + join visual cues
visual_cues_fr = " ".join(clean_lists(visual_cues_fr))

# clean + join dialogue
dialogue_fr = " ".join(clean_lists(dialogue_fr))

# clean + join everything
cleaned_everything_fr = " ".join(clean_lists(orig_toked_fr))

# append to master lists
master_visual_cues.append(visual_cues_fr)
master_dialogue.append(dialogue_fr)
master_setting.append([None])
master_cleaned_everything.append(cleaned_everything_fr)

##### process Green Book

Speaking: all caps NAME \n text
Setting: bold and preceded by ext/int
Character: centered, all caps	
Visual: block of text with some words in all caps for emphasis	
Other Notes: page numbers sandwiched by CONTINUED

##### process Hell or High Water

Speaking: a wall of text
Setting: none, script is just dialogue
Character: none, script is just dialogue
Visual: none, script is just dialogue
Other Notes: page numbers are marked (1/42, 2/42, etc.)

##### process Lady Bird

Speaking: all caps NAME \n text
Setting: preceded by ext/int in all caps	
Character: centered, all caps
Visual: just words	
Other Notes: very little extraneous text at beginning

##### process La La Land

Speaking: all caps NAME \n text
Setting: words	
Character: NAME
Visual: words, sometimes italcized or underlined
Other Notes: music titles are in brackets and bold

##### process Marriage Story

Speaking: all caps NAME \n text
Setting: preceded by ext/int in all caps	centered, all caps	
Character: just words; pattern of character
Visual: name \n dialogue \n free text not preceeded by character name \n and then character name \n or new setting	
Other Notes: N/A

##### process Roma

Speaking: all caps NAME \n text	
Setting: preceded by ext/int in all caps	
Character: centered, all caps	
Visual: block of text	
Other Notes: includes dates in red for design department

##### process Shape of Water

Speaking: all caps NAME \n text, sometimes surrounded by quotes when character Dialogue is speaking	
Setting: preceded by ext/int in all caps	
Character: centered, all caps	
Visual: text, sometimes italicized	
Other Notes: extraneous details at the beginning "for your consideration", etc.

##### process The Big Sick

Speaking: all caps NAME \n text	
Setting: preceded by ext/int in all caps	
Character: centered, all caps	
Visual: just words; pattern of character name \n dialogue \n free text not preceeded by character name \n and then character name \n or new setting	
Other Notes: (V.O.) means voiceover

##### process The Favourite

Speaking: all caps NAME \n text	
Setting: preceded by ext/int in all caps and surrounded by numbers	
Character: centered, all caps	
Visual: preceded by setting and number	
Other Notes: preceding extraneous details

##### process Three Billboards

Speaking: all caps NAME \n text, sometimes words are randomly underlined	
Setting: bold and preceded by ext/int	
Character: bold centered, all caps	
Visual: words	
Other Notes: extraneous details at the beginning "for your consideration", etc.

##### process Vice

Speaking: voiceovers are in quotes but regular dialogue is all caps NAME \n text	
Setting: preceded by ext/int in all caps	
Character: centered, all caps	
Visual: some visual cues are in all caps for emphasis "POLICE LIGHTS FLASH behind him as he drives" and stage directions i.e. (into his ear piece) are in ()
Other Notes: (V.O.) means voiceover

In [ ]:
# make column of dialogue


In [ ]:
# make column of visual cues


In [ ]:
# make column of setting


In [ ]:
# make everything column


In [ ]:
# def remove_consideration(text):
    